In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data= pd.read_csv('D:\\Projects\\Data\\Depression Analysis\\text_emotion.csv') #,encoding='ISO-8859-1', ISO-8859-1
# Keeping only the neccessary columns

In [3]:
data = data[['content','sentiment']]

In [4]:
data.head()

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,negative
2,Funeral ceremony...gloomy friday...,negative
3,wants to hang out with friends SOON!,positive
4,@dannycastillo We want to trade with someone w...,neutral


In [5]:
data = data[data.sentiment != "empty"]
data = data[data.sentiment != "neutral"]
data['content'] = data['content'].apply(lambda x: x.lower())
data['content'] = data['content'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    


31176
29894


In [6]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['content'].values)
X = tokenizer.texts_to_sequences(data['content'].values)
X = pad_sequences(X)

In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           1280000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 32, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 1,535,194
Trainable params: 1,535,194
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(20458, 32) (20458, 2)
(10077, 32) (10077, 2)


In [9]:
print(X_train[0])
print(X_test[0])
print(Y_train[0])
print(Y_test[0])
print(data['sentiment'])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  160 1059 5642    5  504    2    5   48 3582
  596  305]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0  379   30    4   68   31   22  356   35 1059    3
   29  506]
[1 0]
[0 1]
1        negative
2        negative
3        positive
5        negative
6        negative
7        negative
8        negative
9        negative
11       negative
12       negative
13       negative
14       positive
15       negative
16       positive
17       negative
18       negative
19       negative
20       negative
21       positive
23       negative
24       negative
25       negative
26       negative
27       negative
28       negative
29       negative
30       negative
36       negative
37       negative
40       positive
           ...   
39961    positive
39962    positive
39963    positive
39964    positive
39965    negative
39966    pos

In [10]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
 - 83s - loss: 0.5922 - acc: 0.6763
Epoch 2/7
 - 79s - loss: 0.4889 - acc: 0.7686
Epoch 3/7
 - 78s - loss: 0.4226 - acc: 0.8089
Epoch 4/7
 - 79s - loss: 0.3624 - acc: 0.8437
Epoch 5/7
 - 79s - loss: 0.3156 - acc: 0.8649
Epoch 6/7
 - 79s - loss: 0.2764 - acc: 0.8873
Epoch 7/7
 - 79s - loss: 0.2431 - acc: 0.9014


In [11]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.86
acc: 0.70


In [12]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 73.62204724409449 %
neg_acc 65.71815718157181 %


In [13]:
twt = ['Layin n bed with a headache  ughhhh...waitin on your call...']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
print(np.argmax(sentiment))
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0 7898  298  153   22    4  406 4651
  2679   15   43  285]]
0
negative


In [16]:
syriadata= pd.read_csv('D:\\Projects\\Data\\Depression Analysis\\Syria.csv' ,encoding='ISO-8859-1') #,encoding='ISO-8859-1', ISO-8859-1
# Keeping only the neccessary columns
syriadata['text'] = syriadata['text'].apply(lambda x: x.lower())
syriadata['text'] = syriadata['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))


In [17]:
syriadata.head()

,text
0,amnesty international admits syrian saydnaya ...
1,rt joyce_karam front pages of 2 major arab new...
2,rt cov_bhoy guardian guardian front page faken...
3,realdonaldtrump now is when america is suppose...
4,rt thedanechris syria parttime jihadist partti...


In [18]:
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(syriadata)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=32, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]


[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0 643]]


In [27]:
result=[]
for item in syriadata['text']:
    #vectorizing the tweet by the pre-fitted tokenizer instance
    print(item)
    Ptwt = tokenizer.texts_to_sequences(item)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    Ptwt = pad_sequences(Ptwt, maxlen=32, dtype='int32', value=0)    
    Psentiment = model.predict(Ptwt,batch_size=1,verbose = 2)[0]
    print(item,Psentiment)
    result.append([item,Psentiment])


amnesty international admits syrian  saydnaya  report fabricated entirely in uk httpstco6drisksnrg syria saydnaya
amnesty international admits syrian  saydnaya  report fabricated entirely in uk httpstco6drisksnrg syria saydnaya [ 0.37677622  0.62322378]
rt joyce_karam front pages of 2 major arab newspapers tom syria cw massacre defies world tests trump httpstcoa55dk7cvj4
rt joyce_karam front pages of 2 major arab newspapers tom syria cw massacre defies world tests trump httpstcoa55dk7cvj4 [ 0.89185274  0.10814723]
rt cov_bhoy guardian guardian front page fakenews debunked syria httpstcon7xfblsit8
rt cov_bhoy guardian guardian front page fakenews debunked syria httpstcon7xfblsit8 [ 0.89185274  0.10814723]
realdonaldtrump now is when america is supposed to stand strong amp dotherightthing obama was a coward what say httpstcoejamognmsk
realdonaldtrump now is when america is supposed to stand strong amp dotherightthing obama was a coward what say httpstcoejamognmsk [ 0.89185274  0.10814723

rt peterdaou obamas syria policy wasnt perfect but for the trump administration to blame him for an atrocity like this is beyond repr [ 0.89185274  0.10814723]
rt sjredmond somehow i dont think that the last words of the people killed in syria were damn you obama time for potus to do his job
rt sjredmond somehow i dont think that the last words of the people killed in syria were damn you obama time for potus to do his job [ 0.89185274  0.10814723]
rt iv1vi in coordination with assad regime hezbollah smuggled huge amounts of chemical weapons from syria amp stored it in n beqaa of
rt iv1vi in coordination with assad regime hezbollah smuggled huge amounts of chemical weapons from syria amp stored it in n beqaa of [ 0.89185274  0.10814723]
rt ajenglish how chlorine gas became a weapon in syrias civil war httpstcommuxufpzrr httpstco1vdmdsxsve
rt ajenglish how chlorine gas became a weapon in syrias civil war httpstcommuxufpzrr httpstco1vdmdsxsve [ 0.89185274  0.10814723]
rt cov_bhoy guardian

rt americanluvsong susanriceunmasked syria maga syria benghazi realdonaldtrump potus donaldjtrumpjr sebgorka docmurdock jimchil [ 0.89185274  0.10814723]
syria parttime jihadist parttime doctor    httpstcodhoar30l4m
syria parttime jihadist parttime doctor    httpstcodhoar30l4m [ 0.36721578  0.63278425]
realdonaldtrump will do nothing about syria except the usual  lie and blame others  impeachtrump fakepotus stepaside gopfail
realdonaldtrump will do nothing about syria except the usual  lie and blame others  impeachtrump fakepotus stepaside gopfail [ 0.89185274  0.10814723]
rt michaelcastner with realdonaldtrump its always somebody elses faultresponsibility trumprussia syria httpstcoeguu5x3sjd
rt michaelcastner with realdonaldtrump its always somebody elses faultresponsibility trumprussia syria httpstcoeguu5x3sjd [ 0.89185274  0.10814723]
rt shepnewsteam realdonaldtrump blames assad and obamaadmin for attack in syria and tillerson says russia and iran bear great mo
rt shepnewsteam reald

exposed false flag chemical attack in syria to disrupt geneva talks kevorkalmassian  syrianaanalysis  httpstcon4vaarpbdb [ 0.46360108  0.53639895]
rt peterdaou obamas syria policy wasnt perfect but for the trump administration to blame him for an atrocity like this is beyond repr
rt peterdaou obamas syria policy wasnt perfect but for the trump administration to blame him for an atrocity like this is beyond repr [ 0.89185274  0.10814723]
rt abdussamad_aif whats happening in syria is painful to witness   ya allah make it easy for those suffering amp grant jannah to those w
rt abdussamad_aif whats happening in syria is painful to witness   ya allah make it easy for those suffering amp grant jannah to those w [ 0.89185274  0.10814723]
rt peterdaou obamas syria policy wasnt perfect but for the trump administration to blame him for an atrocity like this is beyond repr
rt peterdaou obamas syria policy wasnt perfect but for the trump administration to blame him for an atrocity like this is bey

starting with toppling criminal assad amp liberating syria from all kinds of invasion amp winning the freedom amp dignity for its prowess people [ 0.36721578  0.63278425]
rt evan_mcmullin when free world leaders turn a blind eye to human rights tyrants accelerate their unspeakable atrocities syria https
rt evan_mcmullin when free world leaders turn a blind eye to human rights tyrants accelerate their unspeakable atrocities syria https [ 0.89185274  0.10814723]
rt bencdp instead of getting your panties in a twist over pepsi how about you put that energy in condemning the violence in syria fo
rt bencdp instead of getting your panties in a twist over pepsi how about you put that energy in condemning the violence in syria fo [ 0.89185274  0.10814723]
rt thearabsource oped something is not adding up with the idlib chemical weapons attack httpstcoalxv3abwqv syria
rt thearabsource oped something is not adding up with the idlib chemical weapons attack httpstcoalxv3abwqv syria [ 0.89185274  0.1

...

rt lsarsour the images out of syria are horrific but so were the images for the past 5 years we turned a blind eye we have become num [ 0.89185274  0.10814723]
pray for syria   
pray for syria    [ 0.02115445  0.97884548]
rt teh_021 turkey summons norway envoy over couplinked asylum seekers httpstcobccdjmw15i iran syria ksa usa uk httpstco
rt teh_021 turkey summons norway envoy over couplinked asylum seekers httpstcobccdjmw15i iran syria ksa usa uk httpstco [ 0.89185274  0.10814723]
rt oulosp my article on what does not add up in the idlib chemical weapons attack  httpstcox9vojp9cns syria
rt oulosp my article on what does not add up in the idlib chemical weapons attack  httpstcox9vojp9cns syria [ 0.89185274  0.10814723]
if you like the tshirt
equalpayday syria diewithyou healabrokenheartby philsimms muneve atlus tidal ewgc
httpstcof7ged07zjw
if you like the tshirt
equalpayday syria diewithyou healabrokenheartby philsimms muneve atlus tidal ewgc
httpstcof7ged07zjw [ 0.39086604  0.609133

rt bbcworld un reports chemical weapons used in idlib syria especially on civilians are extremely alarming amp disturbing

httpst [ 0.89185274  0.10814723]
rt lsarsour the images out of syria are horrific but so were the images for the past 5 years we turned a blind eye we have become num
rt lsarsour the images out of syria are horrific but so were the images for the past 5 years we turned a blind eye we have become num [ 0.89185274  0.10814723]
rt maryamalkhawaja despite horrific crimes committed by syria regime some western politicians talking about assad as part of the soluti
rt maryamalkhawaja despite horrific crimes committed by syria regime some western politicians talking about assad as part of the soluti [ 0.89185274  0.10814723]
the latest revolution and arabian news httpstcowjzjp6fryk thanks to kamran_sarvnaz yousifomranjass purple4sports syria
the latest revolution and arabian news httpstcowjzjp6fryk thanks to kamran_sarvnaz yousifomranjass purple4sports syria [ 0.08062018  

rt bonniekipperman abcpolitics redlines syria chemical weapons attack by assad kills over 100people 4 april 2017 httpstcokrtr [ 0.89185274  0.10814723]
rt bonniekipperman statedept redlines syria 2013 1400 killed by sarin amp world allowed assad to do it again httpstco0azqtnrrox
rt bonniekipperman statedept redlines syria 2013 1400 killed by sarin amp world allowed assad to do it again httpstco0azqtnrrox [ 0.89185274  0.10814723]
rt codeaud pm i call on international obligation from 2013 to fully and finally remove these horrible weapons from syria
httpst
rt codeaud pm i call on international obligation from 2013 to fully and finally remove these horrible weapons from syria
httpst [ 0.89185274  0.10814723]
rt aynrandpaulryan realdonaldtrump whitehouse how is this tweet aging 45
tuesdaymotivation 
syria httpstcol2ftjdze4j
rt aynrandpaulryan realdonaldtrump whitehouse how is this tweet aging 45
tuesdaymotivation 
syria httpstcol2ftjdze4j [ 0.89185274  0.10814723]
rt shahram_tavasol iran 

rt borisjohnson horrific reports of chemical weapons attack in idlib syria incident must be investigated amp perpetrators held to account [ 0.89185274  0.10814723]
rt laurie_holden my heart is broken for the people of syriathis is so tragic a senseless loss of life syria
rt laurie_holden my heart is broken for the people of syriathis is so tragic a senseless loss of life syria [ 0.89185274  0.10814723]
rt maytham956 have you ever seen someone who was exposed to chemical weapons while taking a selfie well it happens only in syria http
rt maytham956 have you ever seen someone who was exposed to chemical weapons while taking a selfie well it happens only in syria http [ 0.89185274  0.10814723]
rt unicef 8yearold hussein from syria lost his leg after a mine exploded but he is determined not to give up on his dreams mineawaren
rt unicef 8yearold hussein from syria lost his leg after a mine exploded but he is determined not to give up on his dreams mineawaren [ 0.89185274  0.10814723]
rt sop

syriagasattack wow these people are so evil those poor people sick children heartbreaking syria needs help how can humankind do that [ 0.36721578  0.63278425]
rt mintpressnews idlib syria is controlled by alqaeda chemicalweapons

httpstcoxyynb9oscs
httpstcolhupg7dcvg
httpstco
rt mintpressnews idlib syria is controlled by alqaeda chemicalweapons

httpstcoxyynb9oscs
httpstcolhupg7dcvg
httpstco [ 0.89185274  0.10814723]
jaketapper nikahang arwacnn ask barackobama this is the direct result of him  leading from behind amp u guys httpstcouy2re3tcmy
jaketapper nikahang arwacnn ask barackobama this is the direct result of him  leading from behind amp u guys httpstcouy2re3tcmy [ 0.81978178  0.18021825]
rt lsarsour the images out of syria are horrific but so were the images for the past 5 years we turned a blind eye we have become num
rt lsarsour the images out of syria are horrific but so were the images for the past 5 years we turned a blind eye we have become num [ 0.89185274  0.10814723]
it 

In [19]:
result.argmax(axis=-1)

1

In [29]:
pd.get_dummies(data['sentiment'])

,anger,boredom,enthusiasm,fun,happiness,hate,love,relief,sadness,surprise,worry
1,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,1,0,0
7,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,0,0,1,0,0
11,0,0,0,0,0,0,0,0,0,0,1
12,0,0,0,0,0,0,0,0,1,0,0


In [28]:
len(result)

10000

In [29]:
result[0]

['amnesty international admits syrian  saydnaya  report fabricated entirely in uk httpstco6drisksnrg syria saydnaya',
 array([ 0.37677622,  0.62322378], dtype=float32)]

In [47]:
Rdf=pd.DataFrame(result)

In [48]:
Rdf.head()

,0,1
0,amnesty international admits syrian saydnaya ...,"[0.376776, 0.623224]"
1,rt joyce_karam front pages of 2 major arab new...,"[0.891853, 0.108147]"
2,rt cov_bhoy guardian guardian front page faken...,"[0.891853, 0.108147]"
3,realdonaldtrump now is when america is suppose...,"[0.891853, 0.108147]"
4,rt thedanechris syria parttime jihadist partti...,"[0.891853, 0.108147]"


In [33]:
result[0][1].argmax(axis=-1)

1

In [49]:
Rdf[2]="NA"

In [52]:
Rdf.head()

,0,1,2
0,amnesty international admits syrian saydnaya ...,"[0.376776, 0.623224]",NA
1,rt joyce_karam front pages of 2 major arab new...,"[0.891853, 0.108147]",NA
2,rt cov_bhoy guardian guardian front page faken...,"[0.891853, 0.108147]",NA
3,realdonaldtrump now is when america is suppose...,"[0.891853, 0.108147]",NA
4,rt thedanechris syria parttime jihadist partti...,"[0.891853, 0.108147]",NA


In [57]:
depression=[]
for index, row in Rdf.iterrows():
    print(row[1])
    if(np.argmax(row[1]) == 0):
        print("negative")
        depression.append("negative")
    elif (np.argmax(row[1]) == 1):
        print("positive")
        depression.append("positive")

[ 0.37677622  0.62322378]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.02835219  0.97164786]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.39086604  0.60913396]
positive
[ 0.39086604  0.60913396]
positive
[ 0.02835219  0.97164786]
positive
[ 0.89185274  0.1081

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.0828196   0.91718042]
positive
[ 0.02115445  0.97884548]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.40217167  0.59782839]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.37677622  0.62322378]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.1081

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.26504993  0.73495007]
positive
[ 0.56579667  0.43420336]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.1081

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.40217167  0.59782839]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.39086604  0.60913396]
positive
[ 0.89185274  0.10814723]
negative
[ 0.46360108  0.53639895]
positive
[ 0.02115445  0.97884548]
positive
[ 0.89185274  0.1081

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.08062018  0.91937983]
positive
[ 0.49760315  0.50239688]
positive
[ 0.89185274  0.10814723]
negative
[ 0.39086604  0.60913396]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.46360108  0.53639895]
positive
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.25521168  0.74478829]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.22916847  0.77083153]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.1081

...

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.06595562  0.93404442]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.43452168  0.5654

negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.40217167  0.59782839]
positive
[ 0.54292786  0.4570722 ]
negative
[ 0.39086604  0.60913396]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.39086604  0.60913396]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.54292786  0.4570722 ]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.40217167  0.59782839]
positive
[ 0.40217167  0.59782839]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.25405297  0.74594706]
positive
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.8918527

negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.54292786  0.4570722 ]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.25521168  0.74478829]
positive
[ 0.40217167  0.59782839]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.36721578  0.63278425]
positive
[ 0.89185274  0.10814723]
negative
[ 0.06595562  0.93404442]
positive
[ 0.89185274  0.10814723]
negative
[ 0.26504993  0.73495007]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.8918527

[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.37677622  0.62322378]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.36721578  0.63278425]
positive
[ 0.08062018  0.91937983]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.08062018  0.91937983]
positive
[ 0.37677622  0.62322378]
positive
[ 0.89185274  0.10814723]
negative
[ 0.81978178  0.18021825]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.1081

[ 0.81978178  0.18021825]
negative
[ 0.89185274  0.10814723]
negative
[ 0.39086604  0.60913396]
positive
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative
[ 0.89185274  0.10814723]
negative


In [58]:
len(depression)

10000

In [59]:
Rdf[2]=depression

In [60]:
Rdf.head()

,0,1,2
0,amnesty international admits syrian saydnaya ...,"[0.376776, 0.623224]",positive
1,rt joyce_karam front pages of 2 major arab new...,"[0.891853, 0.108147]",negative
2,rt cov_bhoy guardian guardian front page faken...,"[0.891853, 0.108147]",negative
3,realdonaldtrump now is when america is suppose...,"[0.891853, 0.108147]",negative
4,rt thedanechris syria parttime jihadist partti...,"[0.891853, 0.108147]",negative


In [61]:
Rdf.to_csv("D:\\Projects\\Data\\Depression Analysis\\Syriaoutput.csv")